In [7]:
import random
import numpy as np


class Item: 
    def __init__(self, weight, value):
        self.weight = weight
        self.value = value
        self.in_bag = False
        
    def bag_action(self):
        self.in_bag = not(self.in_bag)
        
    def get_weight(self):
        return self.weight
    
    def get_status(self):
        return self.in_bag
    
    def get_value(self):
        return self.value
    
def print_in_bag(items):
    result = []
    
    for item in items:
        if item.in_bag:    
            result.append(1)
        else:
            result.append(0)
    
    print(result)
    
    
def random_assignment_exp(items, weight_limit, max_diff):
    
    diff = max_diff + 1
    weight = 0
    
    while(diff > max_diff):
        index = random.randint(0,len(items)-1)
        
        if not weight + items[index].get_weight() > weight_limit:
              weight += items[index].get_weight()
              items[index].bag_action()
              
        diff = weight_limit - weight
        

def sum_items(items):
    """
    returns the current value and weight of the knapsack
    """
    
    values = 0
    weights = 0
    
    for item in items:
        if item.in_bag:
            weights += item.get_weight()
            values += item.get_value()
            
    return weights, values


def random_assignment(items, weight_limit):
    """
    randomly puts items in the knapsack until the weight restriction is not violated
    50/50 chance for every items to end up in the bag
    """
    
    for item in items:
        if item.get_status():
            item.bag_action()
    
    while True:
        print("blabla")
        for item in items:
            if random.randint(0,100) > 50:
                item.bag_action()
        
        if weight_limit > sum_items(items)[0]:
            break
    
    return items

        
def small_nbh(items,weight_limit, FC):
    """
    items: list of items
    weight_limit: constraint
    FC: True : first choice hc 
        False : normal hc
    
    Nbh: one item is put in the knapsack. atmost n steps.
    
    Example:
    
        List : 0 0 0
        Nbh  : 1 0 0
               0 1 0
               0 0 1
    
    returns the (updated) item list and a boolean which is True if the list has changed, False otherwise
    """
    
    # collect values and candidates for later evalutation
    value_list = []
    candidate_list = []
    
    # append current value for comparison
    value_list.append(sum_items(items)[1])
    candidate_list.append(-1)
     
    
    for i in range(len(items)):
        
        # put item i in bag, if weight restriction is not violated:
        # collect value and candidate
        
        items[i].bag_action()
        weights, values = sum_items(items)
        
        if not weights > weight_limit:
            value_list.append(values)
            candidate_list.append(i)
            
            # in case of First Choice Hill Climbing we just return the first assignment that improves the value
            if FC and values> value_list[0]:
                return items,True
            else:    
                items[i].bag_action()
        
        else:
            items[i].bag_action()
 
    
    if candidate_list[value_list.index(max(value_list))] == -1:
        # no better asignment found
        return items, False
    else:
        # there is a better assignment, update list and return
        items[candidate_list[value_list.index(max(value_list))]].bag_action()
        return items, True
    
def large_nbh(items, weight_limit, FC): 
    """
    items: list of items
    weight_limit: constraint
    FC: True : first choice hc 
        False : normal hc
    
    Nbh: The status of two items changes (n^2 steps), possible actions:
            put two items in bag,
            take one item out, put one in
            take two out (never occurs)
        
    
    Example:
    
        List : 1 0 
        Nbh  : 0 1
               1 1
               0 0
               1 0 
              
    returns the (updated) item list and a boolean which is True if the list has changed, False otherwise
    """
    
    # collect values and candidates for later evalutation
    value_list = []
    candidate_list = []
    
    value_list.append(sum_items(items)[1])
    candidate_list.append((0,0))
    
    
    for i in range(len(items)):
        
        items[i].bag_action()
        
        if not weight_limit < sum_items(items)[0]:
                
            for j in range(len(items)):
                
                items[j].bag_action()
                
                if weight_limit < sum_items(items)[0]:
                    # assignment is invalid
                    items[j].bag_action()
                else:  
                    # collect only valid values
                    value = sum_items(items)[1]
                    
                    if value > value_list[0]:
                        # append only values that are greater than the initial value
                        if FC:
                            # in FC case, return improved assignment
                            return items, True
                        else:
                            value_list.append(value)
                            candidate_list.append((i,j))
                        
                    items[j].bag_action()
                
        items[i].bag_action()
       
    max_value = max(value_list)
    before = value_list[0]
    
    if candidate_list[value_list.index(max(value_list))] == (0,0):
        # no better assignment found
        return items, False
    else:
        take,put = candidate_list[value_list.index(max(value_list))]
        items[take].bag_action()
        items[put].bag_action()
        return items, True
    
                
def hill_climbing(items, weight_limit, nbh_function,FC):
    """
    performs either First Choice or normal Hill Climbing
    depending on the neighbourhood function with a small or large nbh
    """
    
    changed = True
    
    i = 0
    
    while changed:
        
        i += 1
        
        items, changed = nbh_function(items,weight_limit,FC)
        
    print("done hc")
    return items


def experiment(items_num, exp_num):
    
    wl = 35
    
    fc_values_small = []
    fc_values_large = []
    normal_values_small = []
    normal_values_large = []
    
    test_l = []
    llist = []
    
    for exp in range(exp_num):
        
        for i in range(items_num):
            test_l.append(Item(random.randint(5,15),1*random.randint(0,20)))
            llist.append(test_l[i].get_value())
        
        test_l = random_assignment(test_l,wl)
        print(" weights and values ", sum_items(test_l))
        
        
        print("Items assigned")
        
        # normal, small nbh
        for i in range(exp_num):
            print(i,1)
            test_l = hill_climbing(test_l,wl,small_nbh,False)
            normal_values_small.append(sum_items(test_l)[1])
        
        print("First exp")
        
        # normal, large nbh
        for i in range(exp_num):
            print(i,2)
            test_l = hill_climbing(test_l,wl,large_nbh,False)
            normal_values_large.append(sum_items(test_l)[1])
        
        print("Second exp")
        
        # fc, small nbh
        for i in range(exp_num):
            print(i,3)
            test_l = hill_climbing(test_l,wl,small_nbh,True)
            fc_values_small.append(sum_items(test_l)[1])
        
        print("Third exp")
        
        # fc, large nbh
        for i in range(exp_num):
            print(i,4)
            test_l = hill_climbing(test_l,wl,large_nbh,True)
            fc_values_large.append(sum_items(test_l)[1])
        
        print("Forth exp",exp)
        
    print("normal, small nbh ", np.mean(normal_values_small))
    
    print("normal, large nbh ", np.mean(normal_values_large))
    
    print("fc, small nbh ", np.mean(fc_values_small))
    
    print("fc, large nbh ", np.mean(fc_values_large))
    
#experiment(7,10)
    
fc_values_large = []    
test_l = []
llist = []
wl = 35

for i in range(10):
    
    for i in range(7):
        test_l.append(Item(random.randint(5,15),1*random.randint(0,20)))
        llist.append(test_l[i].get_value())
    
    test_l = random_assignment(test_l,wl)
    print(" weights and values ", sum_items(test_l))
    
    test_l = hill_climbing(test_l,wl,large_nbh,True)
    fc_values_large.append(sum_items(test_l)[1])
    
        
print(fc_values_large)

blabla
 weights and values  (0, 0)
done hc
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
 weights and values  (28, 39)
done hc
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
bl

blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla

blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla

blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla

blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla

blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla

blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla

blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla

blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla
blabla

KeyboardInterrupt: 